In [148]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\MAS GAMING\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [149]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import PIL.Image as Image

Get data

In [153]:
IMAGE_SIZE = (28, 28)
EPOCH = 3
LEARNING_RATE = 0.001

# Import data
data = pd.read_csv(r"../digit-recognizer/train.csv")
data_test = pd.read_csv(r"../digit-recognizer/test.csv")
print(data_test)

# Get labels and image array from data
labels : np.ndarray = data.values[:, 0]
images : np.ndarray = data.values[:, 1:].astype('uint8')

labels_test : np.ndarray = data_test.values[:, 0]
images_test : np.ndarray = data_test.values[:, 1:].astype('uint8')

       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
27995       0       0       0       0       0       0       0       0       0   
27996       0       0       0       0       0       0       0       0       0   
27997       0       0       0       0       0       0       0       0       0   
27998       0       0       0       0       0       0       0       0       0   
27999       0       0       0       0       0       0       0       0       0   

       pixel9  ...  pixel77

Process data

In [151]:
class Model(nn.Module):
    def __init__(self, input, hidden, output):
        super().__init__()
        self.layer1 = nn.Linear(input, hidden)
        self.layer2 = nn.Linear(hidden, output)

    def forward(self, x):
        x = self.layer1(x)
        x = F.sigmoid(self.layer2(x))
        # torch.sigmoid(x)

        return x

In [152]:
model = Model(IMAGE_SIZE[0] * IMAGE_SIZE[1], 1024, 10)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
for i in range(EPOCH):
    wrong = 0
    for j in range(len(images)):
        optimizer.zero_grad()
        res = model(torch.tensor(images[j], dtype=torch.float32))
        # print(res)
        if (int(torch.argmax(res)) != labels[j]):
            wrong += 1

        # Create label for calculating loss
        label_tensor = np.zeros(10)
        label_tensor[labels[j]] = 1
        label_tensor = torch.tensor(label_tensor, dtype=torch.float32)
        # print(label_tensor)
        loss = criterion(res, label_tensor)
        loss.backward()
        optimizer.step()
        
        if ((j+1) % 2000 == 0):
            print(f"Data {j+1}: Wrong = {wrong}, Accuracy: {100-wrong/j*100}%")
        
    print(f"Epoch: {i} --> Wrong: {wrong}, Accuracy: {100-wrong / labels.size * 100}%\n")    



# Test model
# wrong = 0
# for i in range(len(images_test)):
#     res = model(torch.tensor(images_test[i], dtype=torch.float32))
#     if (int(torch.argmax(res)) != labels_test[i]):
#         wrong += 1

# print(f"Accuracy on test data: {100-wrong/labels_test.size*100}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x783 and 784x1024)